In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install antspyx

In [ ]:
import ants
import numpy as np
import os
import nibabel as nib
import pandas as pd
import matplotlib.pyplot as plt
from skimage.util import montage

In [ ]:
def fixRuntimeError(img_path):
  ## https://icode.best/i/53034344111927?fbclid=IwAR1g3e8pwGrN6TKaFQOWqtunBTpytB3NYgEJrMyTLaPvSTU4YMsB2U6XtHM
  print("fixed_img Error"+img_path)
  img = nib.load(img_path)
  qform = img.get_qform()
  img.set_qform(qform)
  sfrom = img.get_sform()
  img.set_sform(sfrom)
  nib.save(img, "/tmp/tmp.nii.gz")
  img = ants.image_read("/tmp/tmp.nii.gz")
  return img

In [ ]:
def ScaleIntensity(antsImage, level=40, width=350):
  array = antsImage.numpy()
  tmp = np.copy(array)
  min_ = level - (width//2)
  max_ = level + (width//2)
  array[tmp > max_] = max_
  array[tmp < min_] = min_
  return antsImage.new_image_like(array) ## create another image with same properties but different data

In [ ]:
def makedirs(path):
  if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
def ants2nib(ants_img):
  ants.image_write(ants_img, "/tmp/tmp.nii.gz")
  nib_img = nib.load("/tmp/tmp.nii.gz")
  return nib_img

def map_coordinate(src_shape, src_affine, dst_shape, dst_affine):
  # Construct coordinate matrix on target space
  coo = np.indices(dst_shape)
  dummy = np.ones([1, *dst_shape])

  # Compute coordinates mapping between source and target
  coo = np.concatenate([coo, dummy], axis=0).reshape([4, -1])
  coo = np.matmul(dst_affine, coo)
  coo = np.linalg.solve(src_affine, coo).reshape([4, *dst_shape])

  # Remove extra ones & round to integers
  coo = np.round(coo[0:3]).astype(np.int32)
  return coo

def interpolate(coo, data):
  # Create empty output
  out = np.zeros(coo.shape[1:], dtype=np.int8)
  # Compute flatten indices
  coo_map = np.ravel_multi_index((coo[0], coo[1], coo[2]), dims=data.shape, mode="clip")
  # Get data from flatten indices and reshape to target shape
  out = data.reshape([-1])[coo_map].reshape(out.shape)
  return out

def change_label(label):
  print(label.shape)
  print(np.unique(label))
  for z in range(label.shape[2]):
    for y in range(label.shape[1]):
      for x in range(label.shape[0]):
        if label[x, y, z] == 2.0:
          label[x, y, z] = 1.0
  print(np.unique(label))
  return label

In [ ]:
def fixing_shape(img, label):
  print("fixRuntimeError:",img)
  nib_img = ants2nib(img)
  nib_label = ants2nib(label)

  coo = map_coordinate(nib_label.shape, nib_label.affine, nib_img.shape, nib_img.affine)
  out = interpolate(coo, np.asanyarray(nib_label.dataobj)).astype(np.uint8)

  label = change_label(out)

  label = nib.Nifti1Image(label, nib_img.affine)
  nib.save(label, "/tmp/tmp.nii.gz")
  label_img = ants.image_read("/tmp/tmp.nii.gz")

  return label_img

In [ ]:
def dilate(image, label):
  ## Dilate the segmentation (radius可調) ##
  # dilated_label = ants.morphology(label, operation='dilate', radius=15, mtype='binary', value=1)
  dilated_label = label #不dilate

  ## 將 dilated_label 和 fixed 相乘，只有在 dilated_label == 1 的地方會有值
  dilated_liver_seg = np.zeros(dilated_label.shape)
  dilated_liver_seg[dilated_label.numpy() == 1] = 1
  # dilated_liver_seg[dilated_label.numpy() == 2] = 2
  filtered_liver = image.new_image_like(image.numpy() * dilated_liver_seg) #image.new_image_like將影像轉為ants格式

  liver_bg = np.ones(dilated_label.shape)*135
  filtered_img_liver = liver_bg * (dilated_liver_seg-1)
  filtered_img_liver = image.new_image_like(filtered_img_liver)

  filtered_img_liver = filtered_img_liver + filtered_liver

  return filtered_img_liver, dilated_label

In [ ]:
def show_and_save(img, img_path, mphase):
  fig, ax = plt.subplots(1, 1, figsize = (50,50))
  ax.imshow(montage(img[:,:,:]), cmap ='gray')
  plt_path = os.path.join(save_filepath, "plt_"+mphase)
  makedirs(plt_path)
  plt.savefig(os.path.join(plt_path, img_path.split("/")[-4]+'_'+mphase+'.png'))

In [ ]:
data_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/ori/'
SEG_data_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/only_liver_dilated/'

# save_filepath = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/Reg_result_whole_image/'
# save_filepath = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/Reg_result_only_liver_dilated/'
# save_filepath = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/(model_SEG_label)Reg_result_only_liver_dilated/'
save_filepath = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/(model_SEG_label)Reg_result_only_liver/'

# save_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/whole_image/'
# save_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/only_liver_dilated/'
# save_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/(model_SEG_label)only_liver_dilated/'
save_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/(model_SEG_label)only_liver/'

image_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/'
mask_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/'
excel_path = '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/classifier/'

txt_filepath = save_filepath+"metric_SyN.txt"
saveError_filepath = save_filepath+"RuntimeError.txt"

makedirs(save_filepath)
makedirs(save_path)

In [ ]:
data = pd.read_excel(excel_path + "all_data.xlsx", sheet_name='img')
ID = data['image'].values

fixing_FL_list = []
moving_FL_list = []
fixed_seg_FL_list = []
moving_seg_FL_list = []
moving_phase = []

for i in range(len(ID)):
  img_PV_path = image_path+ID[i]+"_PV.nii.gz"
  fixing_FL_list.append(img_PV_path)
  PV_path = mask_path+ID[i]+"_PV.nii.gz"
  fixed_seg_FL_list.append(PV_path)

  img_pre_path = image_path+ID[i]+"_pre.nii.gz"
  img_A_path = image_path+ID[i]+"_A.nii.gz"
  moving_FL_list.append([img_pre_path,img_A_path])

  pre_path = mask_path+ID[i]+"_pre.nii.gz"
  A_path = mask_path+ID[i]+"_A.nii.gz"
  moving_seg_FL_list.append([pre_path,A_path])
  moving_phase.append(["pre","A"])

display(fixing_FL_list)
display(fixed_seg_FL_list)
display(moving_FL_list)
display(moving_seg_FL_list)

['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_001_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_002_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_003_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_004_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_005_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_006_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_007_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_009_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_010_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_012_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_013_

['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_001_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_002_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_003_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_004_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_005_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_006_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_007_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_009_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_010_PV.nii.gz',
 '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_012_PV

[['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_001_pre.nii.gz',
  '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_001_A.nii.gz'],
 ['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_002_pre.nii.gz',
  '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_002_A.nii.gz'],
 ['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_003_pre.nii.gz',
  '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_003_A.nii.gz'],
 ['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_004_pre.nii.gz',
  '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_004_A.nii.gz'],
 ['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_005_pre.nii.gz',
  '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_005_A.nii.gz'],
 ['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/

[['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_001_pre.nii.gz',
  '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_001_A.nii.gz'],
 ['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_002_pre.nii.gz',
  '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_002_A.nii.gz'],
 ['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_003_pre.nii.gz',
  '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_003_A.nii.gz'],
 ['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_004_pre.nii.gz',
  '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_004_A.nii.gz'],
 ['/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_phase/HCC_005_pre.nii.gz',
  '/content/drive/MyDrive/Colab_Notebooks/liver_cancer/segmentation/results_p

In [ ]:
# 因為對位的時間較長，為了避免斷線後重複對位，將有對過的紀錄下來，以便下次從中斷點開始跑
ff = open(txt_filepath, 'a')
ff = open(txt_filepath)
exit = []
for line in ff.readlines():
  exit_moving = (line.split(", ")[1]).split(", ")[0]
  print(exit_moving)
  exit.append(exit_moving)
ff.close()
print("exit =",exit)
count = len(exit)-1
print(count)

if len(exit) == 0:
  ff = open(txt_filepath, 'a')
  ff.write("fixing_path, moving_path, MS(fixed_moving), MS(fixed_reg), MS(fixed_moving)-MS(fixed_reg), MI(fixed_moving), MI(fixed_reg), MI(fixed_moving)-MI(fixed_reg)\n")
  ff.close()

moving_path
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_001_pre.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_001_A.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_002_pre.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_002_A.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_003_pre.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_003_A.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_004_pre.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_004_A.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_005_pre.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_005_A.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/data/TCIA/[image]/HCC_006_pre.nii.gz
/content/drive/MyDr

## Registration
https://segmentfault.com/a/1190000041877466

type_of_transform參數的取值可以為：
* Rigid：剛體
* Affine：仿射配準，即剛體+縮放
* ElasticSyN：仿射配準+可變形配準，以MI為優化準則，以elastic為正則項
* SyN：仿射配準+可變形配準，以MI為優化準則
* SyNCC：仿射配準+可變形配準，以CC為優化準則


metric_type : string，image metric to calculate
* MeanSquares
* Correlation
* ANTsNeighborhoodCorrelation
* MattesMutualInformation
* JointHistogramMutualInformation
* Demons

In [ ]:
count = 0

for p_num in range(len(fixing_FL_list)):
  fixed_path = fixing_FL_list[p_num]
  moving_path_list = moving_FL_list[p_num]
  fixed_label_path = fixed_seg_FL_list[p_num]

  for phase_num in range(len(moving_path_list)):
    moving_path = moving_path_list[phase_num]
    if moving_path not in exit:
      print(moving_path)
      moving_label_path = moving_seg_FL_list[p_num][phase_num]
      print(moving_label_path)

      fixed_moving_metric = []
      fixed_reg_metric = []

      try:
        fixed_img = ants.image_read(fixed_path)
      except RuntimeError:
        fixed_img = fixRuntimeError(fixed_path)

      try:
        moving_img = ants.image_read(moving_path)
      except RuntimeError:
        moving_img = fixRuntimeError(moving_path)

      try:
        fixed_label_img = ants.image_read(fixed_label_path)
      except RuntimeError:
        fixed_label_img = fixRuntimeError(fixed_label_path)

      try:
        moving_label_img = ants.image_read(moving_label_path)
      except RuntimeError:
        moving_label_img = fixRuntimeError(moving_label_path)

      ## resample ##
      fixed = ants.resample_image(fixed_img,(1,1,1),0,0)
      moving = ants.resample_image(moving_img,(1,1,1),0,0)
      fixed_label = ants.resample_image(fixed_label_img,(1,1,1),0,1)
      moving_label = ants.resample_image(moving_label_img,(1,1,1),0,1)

      ## Windowing ##
      fixed = ScaleIntensity(fixed)
      moving = ScaleIntensity(moving)

      ## fix label shape (調整為切片數、方向一樣) ##
      label_fixed = fixing_shape(fixed, fixed_label)
      label_moving = fixing_shape(moving, moving_label)

      ## Dilate the segmentation ##
      filtered_fixed_liver, dilated_fixed_label = dilate(fixed, label_fixed)
      filtered_moving_liver, dilated_movinglabel = dilate(moving, label_moving)

      mphase = moving_phase[p_num][moving_path_list.index(moving_path)]

      ## registration ##
      # try:
      mytx = ants.registration(fixed=filtered_fixed_liver, moving=filtered_moving_liver, type_of_transform='SyN') #, mask=label
      warped = mytx['warpedmovout']

      ## save image ##
      reg_path = os.path.join(save_filepath, "HCC_"+fixed_path.split("[image]/")[1].split("_")[1])
      makedirs(reg_path)
      ants.image_write(warped, os.path.join(reg_path, "Reg_"+str(mphase)+".nii.gz"))
      ants.image_write(filtered_fixed_liver, os.path.join(reg_path, "PV.nii.gz"))

      ## Show image ##
      plt.clf()

      plt.figure(figsize=(24,16))
      plt.subplot(331)
      plt.imshow(fixed[:,:,100], cmap='gray')
      plt.title('Fixed(PV)')
      plt.subplot(332)
      plt.imshow(dilated_fixed_label[:,:,100])
      plt.title('Fixed label')
      plt.subplot(333)
      plt.imshow(filtered_fixed_liver[:,:,100], cmap='gray')
      plt.title('Fixed liver')

      plt.subplot(334)
      plt.imshow(moving[:,:,100], cmap='gray')
      plt.title('Moving('+str(mphase)+")")
      plt.subplot(335)
      plt.imshow(dilated_movinglabel[:,:,100])
      plt.title('Moving label')
      plt.subplot(336)
      plt.imshow(filtered_moving_liver[:,:,100], cmap='gray')
      plt.title('Moving liver')

      plt.subplot(337)
      plt.imshow(warped[:,:,100], cmap='gray')
      plt.title('Warped')

      plt_path = os.path.join(save_filepath, "plt")
      makedirs(plt_path)
      plt.savefig(os.path.join(plt_path, "HCC_"+fixed_path.split("[image]/")[1].split("_")[1]+'_'+str(mphase)+'.png'))


      # 顯示所有切面
      show_and_save(filtered_fixed_liver, fixed_path, str(mphase))
      show_and_save(filtered_moving_liver, moving_path, str(mphase))
      show_and_save(warped, moving_label_path, str(mphase))


      # Similarity ##
      metric_MI = ants.image_similarity(filtered_fixed_liver,filtered_moving_liver,metric_type='MattesMutualInformation')
      metric = ants.image_similarity(filtered_fixed_liver,filtered_moving_liver,metric_type='MeanSquares')#MattesMutualInformation #MeanSquares
      fixed_moving_metric.append(metric)

      try:
        metric2_MI = ants.image_similarity(filtered_fixed_liver,warped,metric_type='MattesMutualInformation')
      except RuntimeError:
        metric2_MI = 0
        f_error = open(saveError_filepath, 'a')
        f_error.write(moving_path+'\n')
        f_error.close()

      metric2 = ants.image_similarity(filtered_fixed_liver,warped,metric_type='MeanSquares')#MattesMutualInformation #MeanSquares
      fixed_reg_metric.append(metric2)


      # -------- 計算整張影像的similarity -------- #
      ## invtransforms ：fix 到 move的變換矩陣
      # mywarpedlabel = ants.apply_transforms(fixed=label, moving=None, transformlist= mytx['invtransforms'],interpolator='nearestNeighbor')
      # metric_MI = ants.image_similarity(filtered_fixed_liver,filtered_moving_liver,metric_type='MattesMutualInformation',fixed_mask=label, moving_mask=mywarpedlabel)
      # metric = ants.image_similarity(filtered_fixed_liver,filtered_moving_liver,metric_type='MeanSquares',fixed_mask=label, moving_mask=mywarpedlabel)
      # fixed_moving_metric.append(metric)

      # metric2_MI = ants.image_similarity(filtered_fixed_liver,warped,metric_type='MattesMutualInformation',fixed_mask=label, moving_mask=mywarpedlabel)
      # metric2 = ants.image_similarity(filtered_fixed_liver,warped,metric_type='MeanSquares',fixed_mask=label, moving_mask=mywarpedlabel)
      # fixed_reg_metric.append(metric2)
      # -------------------------------------------- #


      f2 = open(txt_filepath, 'a')
      if metric != float("inf") and metric != float("-inf"):
        f2.write(fixed_path+", "+moving_path+", "+str(int(metric))+", "+str(int(metric2))+", "+str(int(metric-metric2))+", "+str(metric_MI)+", "+str(metric2_MI)+", "+str(metric_MI-metric2_MI)+"\n")
      else:
        f2.write(fixed_path+", "+moving_path+", "+str(metric)+", "+str(metric2)+", NAN"+", "+str(metric_MI)+", "+str(metric2_MI)+", "+str(metric_MI-metric2_MI)+"\n")
      f2.close()

      count += 1

    # except RuntimeError:
    #   f_error = open(saveError_filepath, 'a')
    #   f_error.write(moving_path+'\n')
    #   f_error.close()


print("fixed_moving_metric =",fixed_moving_metric)
print("fixed_reg_metric =",fixed_reg_metric)
print("count =",count)

In [ ]:
## merge channel ##
PATHS = os.listdir(save_filepath)

for ID in PATHS:
  file_PATHS = os.path.join(save_filepath, ID)
  print(ID)
  if "HCC" in ID:
    if os.path.isdir(file_PATHS):
      file_name_list = os.listdir(file_PATHS)
      for file_name in file_name_list:
        file_name_PATHS = os.path.join(file_PATHS, file_name)
        print(file_name_PATHS)
        if os.path.basename(file_name) == "PV.nii.gz":
          channel1 = nib.load(file_name_PATHS)
          channel1_img = channel1.get_fdata()
        elif os.path.basename(file_name) == "Reg_A.nii.gz":
          channel2 = nib.load(file_name_PATHS)
          channel2_img = channel2.get_fdata()
        elif os.path.basename(file_name) == "Reg_pre.nii.gz":
          channel3 = nib.load(file_name_PATHS)
          channel3_img = channel3.get_fdata()


      ## channel first ##
      # merge = np.zeros((3,channel1.shape[0],channel1.shape[1],channel1.shape[2]), 'int16')
      # print(merge.shape)
      # merge[0, ...] = channel1_img
      # merge[1, ...] = channel2_img
      # merge[2, ...] = channel3_img

      ## channel last ##
      merge = np.zeros((channel1.shape[0],channel1.shape[1],channel1.shape[2],3), 'int16')
      # merge = np.zeros((channel1.shape[0],channel1.shape[1],channel1.shape[2],2), 'int16') #只merge兩張影像時
      print(merge.shape)
      merge[..., 0] = channel1_img
      merge[..., 1] = channel2_img
      merge[..., 2] = channel3_img
      print(merge.shape)


      ## save image ##
      # merge_channel_path = os.path.join(save_filepath, "merge_channel(A+PV)")
      merge_channel_path = os.path.join(save_filepath, "merge_channel")
      makedirs(merge_channel_path)
      nib.save(nib.Nifti1Image(merge, affine=channel1.affine), os.path.join(merge_channel_path, ID+".nii.gz"))

      ## Show image ##
      plt.clf()

      plt.figure(figsize=(24,10))
      plt.subplot(131)
      plt.imshow(channel1_img[:,:,100], cmap='gray')
      plt.title('Fixed(PV)'+str(channel1_img.shape))
      plt.subplot(132)
      plt.imshow(channel2_img[:,:,100], cmap='gray')
      plt.title('Reg_arterial'+str(channel2_img.shape))
      plt.subplot(133)
      plt.imshow(channel3_img[:,:,100], cmap='gray')
      plt.title('Reg_pre-contrast'+str(channel3_img.shape))

      # plt_path = os.path.join(save_filepath, "plt_merge(A+PV)")
      plt_path = os.path.join(save_filepath, "plt_merge")
      makedirs(plt_path)
      plt.savefig(os.path.join(plt_path, ID+'.png'))


HCC_001
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/(model_SEG_label)Reg_result_only_liver/HCC_001/Reg_pre.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/(model_SEG_label)Reg_result_only_liver/HCC_001/Reg_A.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/(model_SEG_label)Reg_result_only_liver/HCC_001/PV.nii.gz
(400, 400, 180, 3)
(400, 400, 180, 3)
plt
plt_pre
plt_A
HCC_002
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/(model_SEG_label)Reg_result_only_liver/HCC_002/Reg_pre.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/(model_SEG_label)Reg_result_only_liver/HCC_002/Reg_A.nii.gz
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registration/ANTsPy/TCIA/(model_SEG_label)Reg_result_only_liver/HCC_002/PV.nii.gz
(400, 400, 273, 3)
(400, 400, 273, 3)
HCC_003
/content/drive/MyDrive/Colab_Notebooks/liver_cancer/registrati